In [1]:
import keyboard
from pynput.keyboard import Controller as KeyCtrl
from pynput.keyboard import Key

physical_keyboard = KeyCtrl()
buffered_word = ""
current_layout = "en"  # 초기 설정
converted_positions = set()

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import pandas as pd
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt

# Define the allowed characters (26 lowercase + 26 uppercase)
ALPHABET = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
char_to_idx = {char: idx for idx, char in enumerate(ALPHABET)}

def one_hot_encode(letter):
    """Return a 52-dim one-hot vector for a given letter."""
    vec = np.zeros(len(ALPHABET), dtype=np.float32)
    if letter in char_to_idx:
        vec[char_to_idx[letter]] = 1.0
    return vec
class LanguageClassifier(nn.Module):
    def __init__(self, input_size=52, hidden_size=128, num_layers=1):
        super(LanguageClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, lengths):
        # Pack the padded sequence
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_out, (h_n, _) = self.lstm(packed)
        # h_n is of shape (num_layers, batch, hidden_size); we use the last layer’s hidden state
        h_last = h_n[-1]  # shape: (batch, hidden_size)
        out = self.fc(h_last)  # shape: (batch, 1)
        out = self.sigmoid(out)  # Output between 0 and 1
        return out


model = LanguageClassifier(input_size=52, hidden_size=128, num_layers=1)

model.load_state_dict(torch.load("AI/best_model.pt"))

def predict_language(input_str):
    model.eval()
    seq = [one_hot_encode(ch) for ch in input_str if ch in char_to_idx]
    if not seq: return 0.5  # 기본값
    
    seq_tensor = torch.tensor(seq).unsqueeze(0)
    length = torch.tensor([len(seq)])
    
    with torch.no_grad():
        output = model(seq_tensor, length)
    return output.item()

In [3]:
import platform
import os
import ctypes


def switch_layout(target_lang):
    if platform.system() == 'Windows':
        # HKL_US = 0x04090409, HKL_Korean = 0x04120412
        lang_code = 0x04120412 if target_lang == "ko" else 0x04090409
        ctypes.windll.user32.ActivateKeyboardLayout(lang_code, 0)
    elif platform.system() == 'Darwin':
        src = """
        # macOS 입력 소스 전환 코드 (Objective-C)
        """
        os.system(f"osascript -e '{src}'")

In [4]:
from process.KoEnMapper import conv_en2ko, conv_ko2en



# Example usage
result_ko = conv_en2ko("hello")
result_en = conv_ko2en("안녕하세요")
print("Test : ")
print(result_ko, result_en)

Test : 
ㅗ디ㅣㅐ dkssudgktpdy


In [1]:
def process_conversion(new_layout):
    global buffered_word, converted_positions
    
    if new_layout == "ko":
        converted = conv_en2ko(buffered_word)
    if new_layout == "en":
        converted = conv_ko2en(buffered_word)
    
    # 백스페이스 시뮬레이션
    physical_keyboard.press(Key.backspace)
    physical_keyboard.release(Key.backspace)
    
    # 변환 텍스트 입력
    physical_keyboard.type(converted)
    converted_positions.add(len(buffered_word))

In [ ]:
import logging

# 로그 설정
logging.basicConfig(filename='key_event_log.txt', level=logging.INFO, format='%(asctime)s - %(message)s')

THRESHOLD_HIGH = 0.99
THRESHOLD_LOW = 0.01

def on_key_event(e):
    global buffered_word
    
    # 이벤트 종류와 키 정보 로그
    logging.info(f"Key event detected: {e.name} - Event type: {e.event_type}")
    
    if e.event_type == keyboard.KEY_DOWN:
        if e.name == 'backspace':
            # 변환된 영역 수정 방지
            if len(buffered_word) in converted_positions:
                logging.info("Backspace pressed, but conversion area is locked.")
                return False
            buffered_word = buffered_word[:-1]
            logging.info(f"Backspace pressed. Buffered word: {buffered_word}")
            return  # Ensure return after handling backspace
            
        # Check if the key is a single alphabetic character
        if len(e.name) != 1 or not e.name.isalpha():
            buffered_word = ""
            logging.info(f"Non-alphabet key pressed. Resetting buffered word: {buffered_word}")
            return  # Block further processing for non-alphabet keys
            
        buffered_word += e.name
        logging.info(f"Added '{e.name}' to buffered word. Current buffered word: {buffered_word}")
        
        prob = predict_language(buffered_word)
        logging.info(f"Predicted language probability: {prob} for word: {buffered_word}")
        
        if current_layout == "en" and prob < THRESHOLD_LOW:
            logging.info("Switching to Korean layout")
            process_conversion("ko")
            switch_layout("ko")
        elif current_layout == "ko" and prob > THRESHOLD_HIGH:
            logging.info("Switching to English layout")
            process_conversion("en")
            switch_layout("en")

In [ ]:
# 키보드 리스너 시작
keyboard.hook(on_key_event)
keyboard.wait('f10')  # F10을 누르면 종료